In [4]:
import random
import itertools as it
from tqdm import tqdm

class Vector:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z

    def __add__(self, other):
        if isinstance(other, Vector):
            # Add the corresponding components of the two vectors
            result_x = self.x + other.x
            result_y = self.y + other.y
            result_z = self.z + other.z
            return Vector(result_x, result_y, result_z)
        else:
            raise ValueError("Can only add two Vector objects together")

    def __sub__(self, other):
        if isinstance(other, Vector):
            # Subtract the corresponding components of the two vectors
            result_x = self.x - other.x
            result_y = self.y - other.y
            result_z = self.z - other.z
            return Vector(result_x, result_y, result_z)
        else:
            raise ValueError("Can only subtract two Vector objects")

    def __mul__(self, scalar):
        # Multiply the vector by a scalar
        result_x = self.x * scalar
        result_y = self.y * scalar
        result_z = self.z * scalar
        return Vector(result_x, result_y, result_z)

    def cross(self, other):
        # Compute the cross product of two vectors
        result_x = self.y * other.z - self.z * other.y
        result_y = self.z * other.x - self.x * other.z
        result_z = self.x * other.y - self.y * other.x
        return Vector(result_x, result_y, result_z)

    def dot(self, other):
        # Compute the dot product of two vectors
        return self.x * other.x + self.y * other.y + self.z * other.z
    
    def norm2(self):
        return self.x * self.x + self.y * self.y + self.z * self.z
    
    def norm(self):
        return sqrt(self.norm2())
    
    def reduce(self):
        d = gcd(gcd(self.x,self.y),self.z)
        self.x,self.y,self.z = self.x/d,self.y/d,self.z/d

    def __str__(self):
        return f"({self.x}, {self.y}, {self.z})"

def generate_random_vectors(num_points=8, min_coord=-10, max_coord=10):
    """
    Generate a list of random 3D vectors.

    Args:
    num_points (int): Number of random vectors to generate (default is 8).
    min_coord (int): Minimum coordinate value for each axis (default is -10).
    max_coord (int): Maximum coordinate value for each axis (default is 10).

    Returns:
    list: A list of tuples representing random 3D points.
    """
    points = []
    for _ in range(num_points):
        x = random.randint(min_coord, max_coord)
        y = random.randint(min_coord, max_coord)
        z = random.randint(min_coord, max_coord)
        points.append(Vector(x, y, z))
    return points

def split_by_point_normal(point, normal, vectors):
    """
    Partitions vectors by the plane normal to normal through point.
    It should be at least sqrt(tol) away from the plane to be counted.
    Returns two lists of indices.
    """
    nn2 = normal.norm2()
    assert nn2>0, f"Norm 0"
    Positive = []
    Zero = []
    Negative = []
    for i,v in enumerate(vectors):
        ndv = normal.dot(v) - normal.dot(point)
        if ndv*ndv == 0:
            Zero.append(i)
        elif ndv > 0:
            Positive.append(i)
        else:
            Negative.append(i)
    return Negative, Zero, Positive

def make_choices(N,Z,P):
    """
    Takes elments from Z and adds them to N and P so that they have the same size
    """
    assert (len(P)+len(Z)+len(N))%2==0, "Need an even number of total elements."
    if len(N)+len(Z)<len(P) or len(P)+len(Z)<len(N): #Not possible to distribute evenly
        return
    num_negs = (len(P)+len(Z)-len(N))//2 #How many should be added to N
    for new_negs in it.combinations(Z,num_negs):
        newN=N.copy()
        newP=P.copy()
        for i in Z:
            if i in new_negs:
                newN.append(i)
            else:
                newP.append(i)
        yield newN,newP
        
def print_vectors(vectors,as_list=False):
    if as_list:
        res='['
        for v in vectors:
            res += f'{v}, '
        print(res[:-2]+']')
    else:
        for i,v in enumerate(vectors):
            print(f'{i}: {v}')

def print_partition(partition):
    for N,P,x,n,Z in partition:
        print(f'Plane defined by {Z},')
        print(f'orthogonal to {n} through {x} separates')
        print(f'{N} and {P}')

def find_321_partition(vectors):
    for a,b,c in it.combinations(vectors, 3):
        normal1 = (b-a).cross(c-a)
        normal1.reduce()
        N1,Z1,P1 = split_by_point_normal(a,normal1,vectors)
        #print(N1,P1)
        assert len(Z1)==3, f"Points not in general possition: {Z1}"
        for newN1,newP1 in make_choices(N1,Z1,P1):
            #Aquí ya sé que newP1 y newN1 tienen el mismo tamaño
            #print(' ',newN1,newP1)
            for d,e in it.combinations(vectors, 2):
                normal2 = normal1.cross(e-d)
                normal2.reduce()
                N2,Z2,P2 = split_by_point_normal(d,normal2, vectors)
                #print('  ',N2,P2)
                assert len(Z2)==2, f"Points not in general possition: {Z1}, {Z2}"
                for newN2,newP2 in make_choices(N2,Z2,P2):
                    #Aquí ya sé que newP2 y newN2 tienen el mismo tamaño
                    #print('   ',newN2,newP2)
                    #Hay que revisar si parten en 4 iguales
                    cuadrant1 = {v for v in newN1 if v in newN2}
                    if len(cuadrant1)!=len(vectors)//4:
                        continue
                    cuadrant2 = {v for v in newN1 if v not in newN2}
                    cuadrant3 = {v for v in newP1 if v in newN2}
                    normal3 = normal1.cross(normal2)
                    normal3.reduce()
                    for f in vectors:
                        N3,Z3,P3 = split_by_point_normal(f,normal3, vectors)
                        #print('    ',N3,P3)
                        assert len(Z3)==1, f"Points not in general possition: {Z1}, {Z2}, {Z3}"
                        for newN3,newP3 in make_choices(N3,Z3,P3):
                            #print('     ',newN3,newP3)
                            #Hay que revisar si parten en 8 iguales
                            if len(cuadrant1.intersection(newN3))!=len(vectors)//8 or len(cuadrant2.intersection(newN3))!=len(vectors)//8 or len(cuadrant3.intersection(newN3))!=len(vectors)//8:
                                continue
                            #print("LO ENCONTRÉ!!")
                            return (newN1,newP1,a,normal1,Z1),(newN2,newP2,d,normal2,Z2),(newN3,newP3,f,normal3,Z3)
    return ()

def compute_normals(u1,u2,u3):
    v11=u1.cross(Vector(1,0,0));v12=u1.cross(Vector(0,1,0));
    v21=u2.cross(Vector(1,0,0));v22=u2.cross(Vector(0,1,0));
    v31=u3.cross(Vector(1,0,0));v32=u3.cross(Vector(0,1,0));
    #we want vi1*xi + vi2 to be orthogonal
    # v11 v21 x1 x2 + v11 v22 x1 + v12 v21 x2 + v12 v22 == 0
    # a3 x1 x2 + b3 x1 + c3 x2 + d3 == 0 && a1 x2 x3 + b1 x2 + c1 x3 + d1 == 0 && a2 x3 x1 + b2 x3 + c2 x1 + d2 == 0
    a1 = v21.dot(v31); b1 = v21.dot(v32); c1 = v22.dot(v31); d1 = v22.dot(v32)
    a2 = v31.dot(v11); b2 = v31.dot(v12); c2 = v32.dot(v11); d2 = v32.dot(v12)
    a3 = v11.dot(v21); b3 = v11.dot(v22); c3 = v12.dot(v21); d3 = v12.dot(v22)
    
    disc = (b1*b2*b3 + c1*c2*c3 - a3*b2*d1 - a2*c3*d1 - a1*b3*d2 + a3*c1*d2 + a2*b1*d3 - a1*c2*d3)**2 - 4*(a2*b1*b3 - a1*b3*c2 + a3*c1*c2 - a2*a3*d1)*(-(b2*c3*d1) + c1*c3*d2 + b1*b2*d3 - a1*d2*d3)
    if disc < 0:
        return 0,0,0
    root = sqrt(disc)
    den1 = 2*(a2*b1*b3 - a1*b3*c2 + a3*c1*c2 - a2*a3*d1)
    den2 = 2*(a3*b2*b1 - a2*b1*c3 + a1*c2*c3 - a3*a1*d2)
    den3 = 2*(a1*b3*b2 - a3*b2*c1 + a2*c3*c1 - a1*a2*d3)
    assert den1!=0 and den2!=0 and den3!=0, f"Not in general position {den1,den2,den3}"
    
    x1 = (-(b1*b2*b3) - c1*c2*c3 + a3*b2*d1 + a2*c3*d1 + a1*b3*d2 - a3*c1*d2 - a2*b1*d3 + a1*c2*d3 + root)/den1
    x2 = (-(b1*b2*b3) - c1*c2*c3 - a3*b2*d1 + a2*c3*d1 + a1*b3*d2 + a3*c1*d2 + a2*b1*d3 - a1*c2*d3 + root)/den2
    x3 = (-(b1*b2*b3) - c1*c2*c3 + a3*b2*d1 - a2*c3*d1 - a1*b3*d2 + a3*c1*d2 + a2*b1*d3 + a1*c2*d3 + root)/den3
    #x2 = -(d3 + b3*x1)/(c3 + a3*x1)
    #x3 = -(d2 + c2*x1)/(b2 + a2*x1+1)

    return v11*x1 + v12, v21*x2 + v22, v31*x3 + v32

def find_222_partition(vectors):
    for (a1,b1),(a2,b2),(a3,b3) in it.combinations(it.combinations(vectors, 2),3):
        normal1,normal2,normal3 = compute_normals(b1-a1,b2-a2,b3-a3)
        if normal1 == 0:
            #No partition exists
            continue
        normal1.reduce();normal2.reduce();normal3.reduce()
        N1,Z1,P1 = split_by_point_normal(a1,normal1,vectors)
        N2,Z2,P2 = split_by_point_normal(a2,normal2,vectors)
        N3,Z3,P3 = split_by_point_normal(a3,normal3,vectors)
        assert len(Z1)==2 and len(Z2)==2 and len(Z3)==2, f"Points not in general possition: {Z1}, {Z2}, {Z3}"
        for newN1,newP1 in make_choices(N1,Z1,P1):
            for newN2,newP2 in make_choices(N2,Z2,P2):
                #Hay que revisar si parten en 4 iguales
                cuadrant1 = {v for v in newN1 if v in newN2}
                if len(cuadrant1)!=len(vectors)//4:
                    continue
                cuadrant2 = {v for v in newN1 if v not in newN2}
                cuadrant3 = {v for v in newP1 if v in newN2}
                for newN3,newP3 in make_choices(N3,Z3,P3):
                    #Hay que revisar si parten en 8 iguales
                    if len(cuadrant1.intersection(newN3))!=len(vectors)//8 or len(cuadrant2.intersection(newN3))!=len(vectors)//8 or len(cuadrant3.intersection(newN3))!=len(vectors)//8:
                        continue
                    #print("LO ENCONTRÉ!!")
                    return (newN1,newP1,a1,normal1,Z1),(newN2,newP2,a2,normal2,Z2),(newN3,newP3,a3,normal3,Z3)
    #print("No encontré")
    return ()

def find_partition(vectors):
    partition = find_321_partition(vectors)
    if len(partition)!=0:
        return partition
    partition = find_222_partition(vectors)
    return partition

def points_to_vectors(L):
    return [Vector(x,y,z) for x,y,z in L]

In [5]:
vectors = points_to_vectors([(-70, -21, -62), (52, -35, -59), (87, -88, -94), (-80, 1, 89), (-90, -29, 63), (41, 35, -16), (-45, 3, 84), (70, 27, -7)])
find_partition(vectors)

(([1, 2, 7, 0],
  [3, 4, 6, 5],
  [0, 5]),
 ([1, 5, 7, 6],
  [0, 3, 4, 2],
  [2, 6]),
 ([0, 3, 5, 7],
  [1, 2, 6, 4],
  [4, 7]))

In [2]:
num_points=8
N=10000

totalexamples = 0
counterexamples = 0
for totaltries in tqdm(range(N)):
    vectors = generate_random_vectors(num_points=num_points, min_coord=-100, max_coord=100)
    try:
        partition = find_partition(vectors)
        totalexamples += 1
        if len(partition)==0:
            counterexamples += 1
            print_vectors(vectors,as_list=True)
    except Exception as error:
        continue
print("Examples searched in general position:",totalexamples)
print("Counterexamples found:",counterexamples)
print(100.*counterexamples/totalexamples,'%')

 24%|████████▊                           | 2435/10000 [01:38<2:45:52,  1.32s/it]

[(-96, 81, 78), (31, -14, -18), (-55, -56, 20), (-82, 84, 35), (-80, 28, 64), (-54, -50, 53), (-92, 83, 1), (57, -14, -73)]


 47%|█████████████████                   | 4749/10000 [03:22<2:06:19,  1.44s/it]

[(92, -51, -84), (-13, 84, 32), (42, 76, -45), (-65, 17, 3), (74, 73, -44), (-77, -17, -11), (82, -40, -94), (-20, 55, 56)]


 56%|███████████████████▉                | 5551/10000 [04:15<1:25:26,  1.15s/it]

...

[(75, -86, -2), (-68, 24, 69), (3, -76, -66), (-46, 18, 43), (-13, -12, -96), (-12, 51, -11), (6, 93, -29), (3, -61, -94)]


 87%|█████████████████████████████████     | 8712/10000 [08:03<29:37,  1.38s/it]

[(-66, -21, 22), (19, -64, 55), (13, 48, 25), (72, -7, -74), (31, -96, 85), (-52, -22, 74), (59, 76, -69), (97, 13, -99)]


100%|█████████████████████████████████████| 10000/10000 [08:23<00:00, 19.85it/s]

Examples searched in general position: 9976
Counterexamples found: 6
75/1247 %


In [4]:
num_points=16
N=10000

totalexamples = 0
counterexamples = 0
for totaltries in tqdm(range(N)):
    vectors = generate_random_vectors(num_points=num_points, min_coord=-100, max_coord=100)
    try:
        partition = find_partition(vectors)
        totalexamples += 1
        if len(partition)==0:
            counterexamples += 1
            print_vectors(vectors,as_list=True)
    except Exception as error:
        continue
print("Examples searched in general position:",totalexamples)
print("Counterexamples found:",counterexamples)
print(100.*counterexamples/totalexamples,'%')

100%|███████████████████████████████████| 10000/10000 [2:28:13<00:00,  1.12it/s]

Examples searched in general position: 9874
Counterexamples found: 0
0 %


In [5]:
lista = [Vector(x,x*x,x**3) for x in range(8)]
find_partition(lista)

()

In [6]:
lista = [Vector(x,x*x,x**3) for x in range(-7,8,2)]
find_partition(lista)

()

In [7]:
lista = [Vector(x,x*x,x**3) for x in [-4,-2,-1,0,1,2,3,4]]
p = find_partition(lista)
[print(v) for v in lista]
print_partition(p)

(-4, 16, -64)
(-2, 4, -8)
(-1, 1, -1)
(0, 0, 0)
(1, 1, 1)
(2, 4, 8)
(3, 9, 27)
(4, 16, 64)
Plane defined by [0, 2, 5],
orthogonal to (-6, 3, 1) through (-4, 16, -64) separates
[3, 4, 0, 5] and [1, 6, 7, 2]
Plane defined by [4, 6],
orthogonal to (35, 79, -27) through (1, 1, 1) separates
[2, 3, 7, 4] and [0, 1, 5, 6]
Plane defined by [3],
orthogonal to (-160, -127, -579) through (0, 0, 0) separates
[4, 5, 6, 7] and [0, 1, 2, 3]


In [6]:
num_points=8
N=10000

totalexamples = 0
counterexamples = 0
for totaltries in tqdm(range(N)):
    vectors = generate_random_vectors(num_points=num_points, min_coord=0, max_coord=99)
    try:
        partition = find_partition(vectors)
        totalexamples += 1
        if len(partition)==0:
            counterexamples += 1
            print_vectors(vectors,as_list=True)
    except Exception as error:
        continue
print("Examples searched in general position:",totalexamples)
print("Counterexamples found:",counterexamples)
print(100.*counterexamples/totalexamples,'%')

  8%|███                                  | 822/10000 [00:56<2:42:52,  1.06s/it]

[(22, 87, 25), (29, 62, 67), (27, 78, 21), (69, 29, 54), (6, 78, 26), (63, 39, 50), (1, 54, 48), (50, 31, 72)]


 24%|████████▍                           | 2355/10000 [02:17<4:04:29,  1.92s/it]

[(16, 59, 50), (48, 3, 88), (10, 81, 68), (50, 82, 98), (67, 5, 63), (80, 9, 20), (65, 11, 2), (40, 59, 76)]


 27%|█████████▋                          | 2679/10000 [03:03<3:48:36,  1.87s/it]

...

[(35, 93, 93), (93, 60, 10), (74, 34, 54), (38, 82, 74), (72, 23, 66), (28, 86, 90), (56, 8, 89), (37, 76, 68)]


 82%|███████████████████████████████▎      | 8248/10000 [06:59<33:20,  1.14s/it]

[(51, 55, 88), (79, 83, 8), (66, 6, 74), (61, 93, 25), (58, 29, 87), (90, 72, 20), (28, 94, 91), (55, 86, 28)]


100%|█████████████████████████████████████| 10000/10000 [07:29<00:00, 22.23it/s]

Examples searched in general position: 9892
Counterexamples found: 5
125/2473 %


In [ ]:
vectors = points_to_vectors()
find_partition(vectors)